## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [ ]:
bike_df=pd.read_csv("bikeshare.csv")
print(bike_df.shape)
bike_df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [ ]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [ ]:
bike_df["datetime"]=bike_df.datetime.apply(pd.to_datetime)

- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

In [ ]:
bike_df['year']=bike_df.datetime.apply(lambda x:x.year)
bike_df['month']=bike_df.datetime.apply(lambda x:x.month)
bike_df['day']=bike_df.datetime.apply(lambda x:x.day)
bike_df['hour']=bike_df.datetime.apply(lambda x:x.hour)

- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [ ]:
bike_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2


- datetime 칼럼을 삭제하시오. 또한 casual+registered=count 이므로 casual과 register 칼럼은 따로 필요하지 않으므로 이들도 삭제하시오.

In [ ]:
bike_df.drop(['datetime','casual','registered'],axis=1,inplace=True)

## 2. 선형 회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. 

In [ ]:
y_target=bike_df["count"]
X_features=bike_df.drop(["count"],axis=1,inplace=False)

### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [ ]:
X_features_ohe=pd.get_dummies(X_features,columns=['year','month','day','hour','holiday','workingday','season','weather'])

### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_features_ohe,y_target,test_size=0.3,random_state=0)

### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr_reg=LinearRegression()
lr_reg.fit(X_train,y_train)
pred=lr_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.62072990538732

In [ ]:
lr_reg.intercept_,lr_reg.coef_

(5332551393299.599,
 array([ 2.77000264e+00,  1.85721028e+00, -8.03580732e-01, -7.33464829e-01,
        -3.24723491e+12, -3.24723491e+12, -5.53708738e+11, -5.53708738e+11,
        -5.53708738e+11, -1.68596315e+10, -1.68596315e+10, -1.68596315e+10,
         1.35872304e+12,  1.35872304e+12,  1.35872304e+12, -5.16415820e+11,
        -5.16415820e+11, -5.16415820e+11,  1.82535274e+10,  1.82535275e+10,
         1.82535275e+10,  1.82535275e+10,  1.82535275e+10,  1.82535275e+10,
         1.82535275e+10,  1.82535274e+10,  1.82535275e+10,  1.82535275e+10,
         1.82535275e+10,  1.82535275e+10,  1.82535275e+10,  1.82535275e+10,
         1.82535275e+10,  1.82535275e+10,  1.82535275e+10,  1.82535275e+10,
         1.82535275e+10, -4.28259160e+10, -4.28259160e+10, -4.28259160e+10,
        -4.28259161e+10, -4.28259161e+10, -4.28259160e+10, -4.28259160e+10,
        -4.28259158e+10, -4.28259157e+10, -4.28259158e+10, -4.28259159e+10,
        -4.28259159e+10, -4.28259158e+10, -4.28259158e+10, -4.282591

#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg=SGDRegressor(max_iter=1000,tol=1e-3,penalty=None,eta0=0.1)
sgd_reg.fit(X_train,y_train)

np.sqrt(mean_squared_error(y_test,pred))

101.62072990538732

In [ ]:
sgd_reg.intercept_,sgd_reg.coef_

(array([-4.6354887e+10]),
 array([-2.74248251e+11,  2.68532593e+11, -1.53943887e+11,  1.26694709e+11,
         2.26017266e+10, -6.89566137e+10,  2.50412419e+10, -2.12633621e+11,
         1.83350089e+11,  2.07066644e+11,  7.72020138e+10, -2.36874381e+11,
         1.50023455e+11,  6.12533099e+09, -1.67605965e+11, -5.63321289e+10,
         1.71930883e+11, -1.93648450e+11, -2.74753034e+11,  6.57288379e+09,
        -2.32402387e+11, -1.51901893e+11,  1.18743453e+11, -1.11521814e+11,
         1.74040866e+11,  7.96533885e+10,  2.70998935e+11,  7.54286643e+10,
        -6.88416328e+10,  8.02175032e+10,  5.03321679e+10, -2.48743640e+10,
        -2.30306351e+10, -2.49620723e+09,  1.55819774e+11, -1.78651575e+10,
        -1.50475396e+11,  6.34495785e+09,  1.60792273e+11,  1.60097357e+11,
        -1.10151009e+11,  1.38230084e+11, -6.86723273e+10,  4.65695899e+10,
         2.03689313e+11, -3.49148588e+11, -6.58516469e+10, -5.56553946e+10,
        -1.18283940e+10, -7.51761654e+09, -8.52368025e+09, -9.

#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [ ]:
from sklearn.linear_model import Ridge, Lasso

ridge_reg=Ridge(alpha=0.1)
ridge_reg.fit(X_train,y_train)
pred=ridge_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.61963796032045

In [ ]:
ridge_reg.intercept_,ridge_reg.coef_

(93.59951479800178,
 array([ 2.77635084e+00,  1.85638810e+00, -8.04848466e-01, -7.32755619e-01,
        -4.30350512e+01,  4.30350512e+01, -2.78809868e+01, -1.12490848e+01,
         6.11830405e+00, -9.97383401e+00,  1.52190326e+01,  2.91904860e+00,
        -1.08898027e+01, -1.00814072e+01,  2.46162001e+01,  2.21827929e+01,
         1.88035315e-01, -1.16829806e+00, -9.80464362e+00, -2.81919690e+00,
         1.30083731e+00,  8.68240255e-01,  2.18864828e+00,  4.24340314e-01,
        -7.62089700e+00, -9.53377117e+00,  7.17480447e-02,  1.02674291e+00,
         2.07627610e+00, -5.06884674e+00,  2.13600282e+00,  2.81138083e+00,
         6.74384093e+00,  2.20615475e+00,  1.81487720e+01, -6.33769375e+00,
         1.18206464e+00, -1.24689439e+02, -1.45447914e+02, -1.59260535e+02,
        -1.66295508e+02, -1.68110470e+02, -1.49564140e+02, -9.42788975e+01,
         4.05901985e+01,  1.92968022e+02,  4.08782973e+01, -1.66084925e+01,
         5.55252726e+00,  5.22344891e+01,  4.69859712e+01,  3.274547

In [ ]:
lasso_reg=Lasso(alpha=0.1)
lasso_reg.fit(X_train,y_train)
pred=lasso_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.56225421214636

In [ ]:
lasso_reg.intercept_,lasso_reg.coef_

(218.65240119480862,
 array([ 3.27180766e+00,  1.76978059e+00, -8.73804775e-01, -7.10780296e-01,
        -8.50381823e+01,  0.00000000e+00, -2.96980105e+01, -1.35728022e+01,
         0.00000000e+00, -4.87987262e+00,  1.71401016e+01,  2.85040566e+00,
        -5.27072118e+00, -3.75178267e+00,  3.03745171e+01,  1.90334989e+01,
         0.00000000e+00, -0.00000000e+00, -8.66790918e+00, -1.84686122e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.39095552e+00, -8.72523168e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.85360130e+00,  0.00000000e+00,  1.87183492e-01,
         3.68292429e+00,  0.00000000e+00,  1.48958750e+01, -4.98174542e+00,
         0.00000000e+00, -1.49473706e+02, -1.70097066e+02, -1.83473824e+02,
        -1.90307400e+02, -1.91967177e+02, -1.73459149e+02, -1.18167091e+02,
         1.18118347e+01,  1.63673667e+02,  1.08736427e+01, -4.25399488e+01,
        -2.08513802e+01,  2.03154895e+01,  1.47011803e+01,  1.27447

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd  

## 3. 로지스틱 회귀 분석 실습 _ 와인 종류 분류하기 : 와인의 화학 조성을 사용하여 와인의 종류를 예측하기 위한 데이터이다. 

ㅇ 타겟 데이터 : 와인의 종류 0, 1, 2의 세가지 값

ㅇ 특징 데이터

* 알콜(Alcohol)

* 말산(Malic acid)

* 회분(Ash)

* 회분의 알칼리도(Alcalinity of ash)

* 마그네슘(Magnesium)

* 총 폴리페놀(Total phenols)

* 플라보노이드 폴리페놀(Flavanoids)

* 비 플라보노이드 폴리페놀(Nonflavanoid phenols)

* 프로안토시아닌(Proanthocyanins)

* 색상의 강도(Color intensity)

* 색상(Hue)

* 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)

* 프롤린(Proline)

1. 와인 데이터 가져오기 

In [4]:
from sklearn import datasets
wine_data = datasets.load_wine()

' 데이터 셋을 살펴보는 코드\nprint(wine_data.DESCR)\n'

더 자세한 데이터에 대한 설명을 보고싶으시다면 아래의 코드를 실행해주시면 됩니다. 

In [7]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

#### 3-2. 입력변수와 목표변수 설정하기
* 입력변수를 사용하기 편하게 pandas dataframe으로 변환해주세요.
  - 입력변수는 data (.data) 함수를 써서 가져오시면 됩니다.
  - 칼럼명 가져오기 : .feature_names 
* 목표변수를 사용하기 편하게 pandas dataframe으로 변환해주세요
  - 목표변수는 target (.target) 함수를 써서 가져요오시면 됩니다. 
  - 칼럼명은 'target'으로 지정해주세요

In [5]:
# 입력 변수를 사용하기 편하게 pandas dataframe으로 변환
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# 목표 변수를 사용하기 편하게 pandas dataframe으로 변환
y = pd.DataFrame(wine_data.target, columns=['Y/N'])

#### 3-3. 훈련 데이터, 테스트 데이터 만들기
* train_test_split 함수를 사용해 테스트 셋의 비율이 0.2가 되도록 X_train, X_test, y_train, y_test 을 만들어주세요. random_state=5로 설정해주세요.
* 경고메세지를 없애도록 y_train = y_train.values.ravel() 를 실행해주세요

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)
y_train = y_train.values.ravel() #경고 메시지를 없애기 위한 코드 

#### 3-4. 모델훈련
* 로지스틱회귀 함수에서 옵션을 solver='saga', max_iter=7500으로 설정해주세요.
* 학습데이터로 모델을 학습시켜주세요

In [7]:
# 필요한 라이브러리 import
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(solver='saga', max_iter=7500)  # sci-kit learn에서 로지스틱 모델을 가지고 온다
logistic_model.fit(X_train, y_train)  # 학습 데이터를 이용해서 모델을 학습 시킨다


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=7500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

#### 3-5. 예측
* 테스트 데이터 셋으로 로지스틱 회귀 모델을 이용해 와인 데이터 분류를 예측해 결과를 y_test_predict 변수에 담아주세요
* score함수를 사용해 로지스틱 회귀 모델의 성능을 score 변수에 담아주세요
* y_test_predict와 score 결과를 확인해주세요.

In [8]:

# 로지스틱 회귀 모델를 이용해서 각 와인 데이터 분류를 예측함
y_test_predict = logistic_model.predict(X_test)

# 로지스틱 회귀 모델의 성능 확인 (정확성 %를 리턴함)
score = logistic_model.score(X_test, y_test)
y_test_predict, score

(array([0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2]), 0.75)

#### (선택) 3-6. GridSearchCV 복습

`LogisticRegression` 모델에 `param_grid`를 사용하여 GridSearchCV를 진행해주세요.
- cv = 5
- 추가적으로 코드를 작성하실 필요 없이, 골뱅이@@@ 부분의 코드를 완성해서 출력해주시면 됩니다. 

In [9]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

# param_grid (수정하셔도 됩니다)
params = {
    'penalty':['l1', 'l2', 'elasticnet'],        # l1 is Lasso, l2 is Ridge
    'C': np.linspace(0.00002,1,100)
}

# GridSearch
grid_search = GridSearchCV(LogisticRegression(), param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([2.00000000e-05, 1.0...
       8.48487879e-01, 8.58588687e-01, 8.68689495e-01, 8.78790303e-01,
       8.88891111e-01, 8.98991919e-01, 9.09092727e-01, 9.19193535e-01,
       9.29294343e-01, 9.39395152e-01, 9.49495960e-01, 9.59596768e-01,
       9.69697576e-01, 9.79798384e-01, 9.89899192e-01, 1.00000000e+00]),
      

학습된 gridsearch 모델에 저장되어 있는 best parameter, best score 를 출력해주세요.

In [10]:
grid_search.best_params_, grid_search.best_score_

({'C': 0.07072565656565656, 'penalty': 'l2'}, 0.9652709359605911)